# FInDe

## Availables banks
- Cajamar
- Openbank
- Unicaja

## Fist step
Put your csv or xls files inside ```accounts/current/<bank_name>/file``` and ```accounts/savings/<bank_name>/file```

## Requisites

In [ ]:
!pip install plotly

## Code

### Imports

In [ ]:
import plotly as py
import plotly.graph_objs as go

import numpy as np
import datetime as dt
import os.path
import xlrd
import pandas as pd

py.offline.init_notebook_mode(connected=True)

### Constant and settings

In [ ]:
CURRENT = "current"
SAVING = "saving"
ACCOUNTTYPES  = [CURRENT, SAVING]
SUPPORTED_BANKS = {
    "unicaja": "Unicaja",
    "cajamar": "Cajamar",
    "openbank": "Openbank"
}

### Process CSV and XLS

In [ ]:
def processCSV(file, splitToken, bank): #TODO Move to pandas
    fecha = []
    saldo = []
    first = True
    
    if bank == 'cajamar':
        with open(file) as f:
            for line in f:
                if first is False:
                    tokens = line.split(splitToken)
                    fecha.append(dt.datetime.strptime(tokens[0], '%d/%m/%Y').date())
                    saldo.append(float(tokens[len(tokens)-1][:-1].replace('.', '').replace(',', '.')))
                first = False

    return (saldo, fecha)

In [ ]:
def processXLS(file, bank):   
    balance = []
    date = []
    
    if bank == 'unicaja':
        book = xlrd.open_workbook(file)
        first_sheet = book.sheet_by_index(0)
        date_int = first_sheet.col_values(0)[5:]
        balance = first_sheet.col_values(5)[5:]
        date = [xlrd.xldate_as_datetime(date_int[i], book.datemode).date().isoformat() for i in xrange(0, len(date_int))]
    
    if bank == 'openbank':
        df = pd.read_html(file)
        df = df[0].dropna(axis=0, thresh=4)
        date_str = df.iloc[1:,1]
        date = [dt.datetime.strptime(x, '%d/%m/%Y').date() for x in date_str]
        balance_str = df.iloc[1:,9]
        balance = [float((x[:-2] + ',' + x[-2:]).replace('.','').replace(',','.')) for x in balance_str]

    return (balance, date)

### Load files

In [ ]:
def sortDataFiles(bankdata):
    for accountType in ACCOUNTTYPES:
        for bankname in bankdata[accountType]:
            (bankdata[accountType][bankname]["date"],bankdata[accountType][bankname]["balance"]) =zip(*sorted(zip(bankdata[accountType][bankname]["date"], bankdata[accountType][bankname]["balance"])))
            
    return bankdata

In [ ]:
def loadDataFiles():    
    print("Data files found:")
    bankdata = {}
    for accountType in ACCOUNTTYPES:
        path = "accounts/{}/".format(accountType)
        directories = [f for f in os.listdir(path) if os.path.isdir(path+f)]
        bankdata[accountType] = {}
        for bankname in directories:
            bankdata[accountType][bankname] = {"date":[], "balance":[]}
            files = [os.path.join(path+bankname,f) for f in os.listdir(path+bankname)]
            #files.sort(key=lambda x: os.path.getmtime(x))
            for datafile in files:
                extension = os.path.splitext(datafile)[1]
                if extension == ".xls":
                    print("{} - {}".format(bankname, datafile))
                    (balance, date) = processXLS(datafile, bankname)
                    bankdata[accountType][bankname]["balance"].extend(balance)
                    bankdata[accountType][bankname]["date"].extend(date)
                if extension == ".csv":
                    print("{} - {}".format(bankname, datafile))
                    (balance, date) = processCSV(datafile, '\t', bankname)
                    bankdata[accountType][bankname]["balance"].extend(balance)
                    bankdata[accountType][bankname]["date"].extend(date)
    
    return sortDataFiles(bankdata)

### Traces

In [ ]:
def builtTraces(bankdata):
    data = []
    
    for accountType in ACCOUNTTYPES: # Account type
        for bank in bankdata[accountType]: # Bank name
            dates = np.array(bankdata[accountType][bank]['date'])
            balances = np.array(bankdata[accountType][bank]['balance'])
            
            bankName = SUPPORTED_BANKS[bank]

            trace_main = go.Scatter(
                x = dates,
                y = balances,
                name = bankName + ': Saldo ' + str(format(balances[-1], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
                #line = dict(
                #    color = 'green'
                #),
                mode = 'lines'
            )
            data.append(trace_main)

            trace_max = go.Scatter(
                x = dates,
                y = [balances.max() for f in dates],
                name = bankName + ': Saldo máximo',
                #visible = 'legendonly',
                #hoverinfo = 'name',
                line = dict(
                    #color = 'cyan',
                    width = 4,
                    dash = 'dot'
                )
            )
            data.append(trace_max)

            trace_mean = go.Scatter(
                x = dates,
                y = [balances.mean() for f in dates],
                name = bankName + ': Saldo medio',
                #hoverinfo = 'none',
                line = dict(
                    #color = 'magenta',
                    width = 4,
                    dash = 'dashdot'
                )
            )
            data.append(trace_mean)

            trace_min = go.Scatter(
                x = dates,
                y = [balances.min() for f in dates],
                name = bankName + ': Saldo mínimo',
                line = dict(
                    #color = 'red',
                    width = 4,
                    dash = 'dot'
                )
            )
            data.append(trace_min)
        
    return (data, 0, 0)

### Plot

In [ ]:
bankdata = loadDataFiles() # Load files
(data, minDate, maxDate) = builtTraces(bankdata) # Built traces from data

# Plot charts
layout = go.Layout(title = 'Saldo ' + str(minDate) + ' / ' + str(maxDate),
              xaxis = dict(title = 'Fecha'),
              yaxis = dict(title = 'Saldo (€)'),
              showlegend = True
)

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='styled-line')